## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [5]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201702')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 6.508746385574341 seconds ---
--- 6.439056396484375 seconds ---
--- 6.0142199993133545 seconds ---
--- 6.165328025817871 seconds ---
--- 6.026252031326294 seconds ---
--- 5.981155157089233 seconds ---
--- 7.792041063308716 seconds ---
--- 9.055203437805176 seconds ---
--- 7.822920799255371 seconds ---
--- 7.520376205444336 seconds ---
--- 7.515448570251465 seconds ---
--- 8.57888150215149 seconds ---
--- 6.345381498336792 seconds ---
--- 6.4234113693237305 seconds ---
--- 6.526384353637695 seconds ---
--- 6.459148168563843 seconds ---
101030


In [6]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201703')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 9.55379581451416 seconds ---
--- 10.065983772277832 seconds ---
--- 8.129196405410767 seconds ---
--- 8.099434852600098 seconds ---
--- 9.19154167175293 seconds ---
--- 9.958981990814209 seconds ---
--- 7.7313361167907715 seconds ---
--- 8.358144283294678 seconds ---
--- 8.372468709945679 seconds ---
--- 9.524359464645386 seconds ---
--- 7.730841398239136 seconds ---
--- 7.8709635734558105 seconds ---
--- 9.530184268951416 seconds ---
--- 11.541909456253052 seconds ---
--- 9.67682147026062 seconds ---
--- 8.63088059425354 seconds ---
154527


## Check and split data

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 300000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines > 6 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines > 6 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

## EMBER

In [14]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [9]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [10]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, 100000, valid_sets=valid_sets, early_stopping_rounds=10)
    y_pred = model.predict(x_val)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [11]:
x_etrain = get_ember_feature(x_train)
x_eval = get_ember_feature(x_val)
x_etest = get_ember_feature(x_test)

In [12]:
import matplotlib.pyplot as plt
model = get_model(x_etrain, y_train, x_eval, y_val)

[1]	valid_0's binary_logloss: 0.446362
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.395605
[3]	valid_0's binary_logloss: 0.356458
[4]	valid_0's binary_logloss: 0.324558
[5]	valid_0's binary_logloss: 0.29884
[6]	valid_0's binary_logloss: 0.276706
[7]	valid_0's binary_logloss: 0.257758
[8]	valid_0's binary_logloss: 0.241391
[9]	valid_0's binary_logloss: 0.227203
[10]	valid_0's binary_logloss: 0.214519
[11]	valid_0's binary_logloss: 0.203253
[12]	valid_0's binary_logloss: 0.193508
[13]	valid_0's binary_logloss: 0.184491
[14]	valid_0's binary_logloss: 0.176772
[15]	valid_0's binary_logloss: 0.169711
[16]	valid_0's binary_logloss: 0.163527
[17]	valid_0's binary_logloss: 0.157913
[18]	valid_0's binary_logloss: 0.152591
[19]	valid_0's binary_logloss: 0.147744
[20]	valid_0's binary_logloss: 0.143751
[21]	valid_0's binary_logloss: 0.139774
[22]	valid_0's binary_logloss: 0.136592
[23]	valid_0's binary_logloss: 0.133326
[24]	valid_0's binary_loglos

In [12]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

In [14]:
y_p = model.predict(x_etest)
y_pred_e = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred_e[i, 0] = y_p[i]

estimate_model(y_pred_e, y_test)

loss : 0.20370
auc score : 0.97656
accuracy score : 0.92784
thre: 0.9292815104
fp:  0.0009850342
recall:  0.3704709311


(0.9765633857954636, 0.20369945163005979, 0.3704709311244056)

## Malcon

In [8]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

Using TensorFlow backend.


In [9]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Multiply, GlobalMaxPooling1D, Dropout
from keras.models import load_model
from sklearn.model_selection import train_test_split

class TMalConv(object):
    """
    train of mal conv
    """

    def __init__(self):
        self.max_len = max_length
        self.history = None
        self.model = None
        self.p_md5 = None
        self.time = time.time()
        self.summary = {
            'time':time.time(),
            'batch_size': 32,
            'epochs': 64,
            'g_c_filter': 128,
            'g_c_kernel_size': 500,
            'g_c_stride': 500,
        }

    def run(self):
        """
        :return:
        """
        self.train()
        
    def get_p(self, key):
        """
        get the parameter from the summary
        :param key:
        :return:
        """
        return self.summary[key]

    def gate_cnn(self, gate_cnn_input):
        """
        construct a gated cnn by the specific kernel size
        :param gate_cnn_input:
        :param kernel_size:
        :return:
        """
        conv1_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"))(
            gate_cnn_input)
        conv2_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"),
                           activation="sigmoid")(gate_cnn_input)
        merged = Multiply()([conv1_out, conv2_out])
        gate_cnn_output = GlobalMaxPooling1D()(merged)
        return gate_cnn_output

    def get_model(self):
        """
        get a model
        :param max_len:
        :param kernel_sizes:
        :return:
        """
        net_input = Input(shape=(self.max_len,))

        embedding_out = Embedding(256, 8, input_length=self.max_len)(net_input)
        merged = self.gate_cnn(embedding_out)

        dense_out = Dense(128)(merged)
        
        net_output = Dense(1, activation='sigmoid')(dense_out)

        model = keras.models.Model(inputs=net_input, outputs=net_output)

        return model

    def train(self):
        batch_size = self.get_p("batch_size")
        epochs = self.get_p("epochs")

        self.model = self.get_model()

        print('Length of the train: ', len(x_train))
        print('Length of the validation: ', len(x_val))
        
#         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/BaseTrain/models/"+ str(self.time) +"-{epoch:04d}-{val_loss:.5f}-{val_acc:.5f}.h5"
        early_stopping = EarlyStopping("val_loss", patience=3, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point, early_stopping]

        # Generators
        training_generator = DataGenerator(range(len(x_train)), x_train, y_train, batch_size, self.max_len)
        validation_generator = DataGenerator(range(len(x_val)), x_val, y_val, batch_size, self.max_len)

        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

        self.model.fit_generator(generator=training_generator,
                                 validation_data=validation_generator,
                                 use_multiprocessing=True,
                                 epochs=epochs,
                                 workers=6,
                                 callbacks=callbacks_list)

In [17]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [18]:
t_instance = TMalConv()
t_instance.run()

Length of the train:  90927
Length of the validation:  10103
Epoch 1/64
2070/2841 [====================>.........] - ETA: 11:09 - loss: 0.1928 - acc: 0.9362

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.157023). Check your callbacks.
  % delta_t_median)


2346/2841 [=======================>......] - ETA: 6:54 - loss: 0.1892 - acc: 0.9377

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.178676). Check your callbacks.
  % delta_t_median)


2841/2841 [==============================] - 2778s 978ms/step - loss: 0.1845 - acc: 0.9392 - val_loss: 0.1729 - val_acc: 0.9478

Epoch 00001: val_loss improved from inf to 0.17285, saving model to /home/zhaoqi/BaseTrain/models/1532328542.9507692-0001-0.17285-0.94782.h5
Epoch 2/64
 561/2841 [====>.........................] - ETA: 37:40 - loss: 0.1337 - acc: 0.9567

...

2788/2841 [============================>.] - ETA: 35s - loss: 0.0893 - acc: 0.9745

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.162166). Check your callbacks.
  % delta_t_median)


2814/2841 [============================>.] - ETA: 18s - loss: 0.0892 - acc: 0.9745

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.168777). Check your callbacks.
  % delta_t_median)


2841/2841 [==============================] - 2117s 745ms/step - loss: 0.0894 - acc: 0.9745 - val_loss: 0.1560 - val_acc: 0.9538

Epoch 00005: val_loss did not improve from 0.15592


In [10]:
model_dir = '/home/zhaoqi/BaseTrain/models/'
f_name = '1532328542.9507692-0002-0.15592-0.94990.h5'
c_model = load_model(model_dir + f_name)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
y_pred = c_model.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

4828/4828 [==============================] - 4339s 899ms/step


In [15]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : 0.29392
auc score : 0.96058
accuracy score : 0.91147
thre: 0.9427035451
fp:  0.0008173228
recall:  0.2806996011


(0.9605778832185878, 0.2939165563363736, 0.2806996011046333)

## Merge features

### Merge Malconv and Ember

In [ ]:
from keras.models import Model

model_f = Model(c_model.input, c_model.layers[-2].output)

train_generator = DataGenerator(range(len(x_train)), x_train, y_train, 32, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, y_val, 32, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

 778/4828 [===>..........................] - ETA: 1:05:20

In [ ]:
model_m = get_model(malcon_train_x, y_train[0:len(malcon_train_x)], malcon_val_x, y_val[0:len(malcon_val_x)] )

In [ ]:
y_p = model_m.predict(malcon_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(malcon_test_x)])

In [ ]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 128+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:128] = m_data[index]
        m_x[index, 128:128+2351] = e_data[index]  
    return m_x

In [ ]:
merge_train_x = merge_feature(malcon_train_x, x_etrain)
merge_val_x = merge_feature(malcon_val_x, x_eval)
merge_test_x = merge_feature(malcon_test_x, x_etest)

model_m = get_model(merge_train_x, y_train[0:len(merge_train_x)], merge_val_x, y_val[0:len(merge_val_x)] )

In [ ]:
y_p = model_m.predict(merge_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(merge_test_x)])

### StandardScaler

In [60]:
test = pd.DataFrame(merge_train_x)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478
0,0.705183,1.479086,-1.532787,1.188892,-1.221726,1.624470,-0.073439,-1.389202,-0.050010,1.532788,...,0.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0
1,-0.385240,-1.328830,1.294801,-0.842054,1.094820,-1.422021,0.374867,1.221696,-0.134544,-1.071150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.394131,0.870116,-1.014822,0.788210,-0.986084,1.494880,-0.084611,-1.264098,-0.649978,1.641255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.286791,-0.833753,0.517326,-0.506641,0.325388,-0.763379,0.083963,0.569139,0.049757,-0.532283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.473867,0.949333,-1.038878,0.761097,-0.942002,1.515890,-0.161014,-1.152947,0.230228,1.190743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(merge_train_x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [72]:
s_merge_train_x = scaler.transform(merge_train_x)
s_merge_val_x = scaler.transform(merge_val_x)
s_merge_test_x = scaler.transform(merge_test_x)

model_m = get_model(s_merge_train_x, y_train[0:len(s_merge_train_x)], s_merge_val_x, y_val[0:len(s_merge_val_x)] )

[1]	valid_0's binary_logloss: 0.606546
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.535902
[3]	valid_0's binary_logloss: 0.476737
[4]	valid_0's binary_logloss: 0.426738
[5]	valid_0's binary_logloss: 0.384202
[6]	valid_0's binary_logloss: 0.347401
[7]	valid_0's binary_logloss: 0.315518
[8]	valid_0's binary_logloss: 0.288056
[9]	valid_0's binary_logloss: 0.263826
[10]	valid_0's binary_logloss: 0.242805
[11]	valid_0's binary_logloss: 0.224185
[12]	valid_0's binary_logloss: 0.207686
[13]	valid_0's binary_logloss: 0.193076
[14]	valid_0's binary_logloss: 0.18006
[15]	valid_0's binary_logloss: 0.168389
[16]	valid_0's binary_logloss: 0.157996
[17]	valid_0's binary_logloss: 0.148482
[18]	valid_0's binary_logloss: 0.140271
[19]	valid_0's binary_logloss: 0.132753
[20]	valid_0's binary_logloss: 0.126112
[21]	valid_0's binary_logloss: 0.120228
[22]	valid_0's binary_logloss: 0.11487
[23]	valid_0's binary_logloss: 0.110088
[24]	valid_0's binary_logloss

[199]	valid_0's binary_logloss: 0.0462829
[200]	valid_0's binary_logloss: 0.0462199
[201]	valid_0's binary_logloss: 0.0462193
[202]	valid_0's binary_logloss: 0.0461826
[203]	valid_0's binary_logloss: 0.0461578
[204]	valid_0's binary_logloss: 0.0461028
[205]	valid_0's binary_logloss: 0.046082
[206]	valid_0's binary_logloss: 0.0460718
[207]	valid_0's binary_logloss: 0.0461296
[208]	valid_0's binary_logloss: 0.0461759
[209]	valid_0's binary_logloss: 0.0462082
[210]	valid_0's binary_logloss: 0.0462406
[211]	valid_0's binary_logloss: 0.0462647
[212]	valid_0's binary_logloss: 0.0462464
[213]	valid_0's binary_logloss: 0.0461945
[214]	valid_0's binary_logloss: 0.0461967
[215]	valid_0's binary_logloss: 0.0461552
[216]	valid_0's binary_logloss: 0.0461221
Early stopping, best iteration is:
[206]	valid_0's binary_logloss: 0.0460718
val loss : 0.04607
auc score : 0.99867
accuracy score : 0.98304


In [73]:
y_p = model_m.predict(s_merge_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(s_merge_test_x)])

loss : 0.04807
auc score : 0.99855
accuracy score : 0.98192
thre: 0.9962235704
fp:  0.0008132561
recall:  0.8711143241


(0.998552209126074, 0.048069058625024294, 0.8711143240731178)